In [1]:
import pandas as pd
import ibm_db_dbi as idb
from sqlalchemy import create_engine
import pyodbc
#import os
import warnings
import pyvalidata as pvd
import hashlib
import time 
import datetime
import numpy as np
import sqlalchemy
import ibm_db_sa
import csv

In [3]:
# PIMMART DB Test Connection
server ="21-eastus2-3534-idq-sqlmi.cc4c669dc8d1.database.windows.net"
database="PIMIDQ" 
username="abdus.samad@kroger.com"
Authentication='ActiveDirectoryInteractive'
src_conn_pim = pyodbc.connect(
        "DRIVER={ODBC Driver 17 for SQL Server};SERVER="
        + server
        + ";DATABASE="
        + database
        + ";UID="
        + username
        + ";AUTHENTICATION="
        + Authentication
    )
print("\n\nConnected to PIM Database.")



Connected to PIM Database.


In [34]:
# TLOG ODS Prod Connection
kods_hostname = "u060odsd101.kroger.com"
kods_database = "ODSPROD"
kods_port = "50000"
kods_protocol = "TCPIP"
kods_username = 'SVCSOAT1'
kods_password = 'mKdaL0d7Rv3tq693lTh7wWG'
kods_conn_str = "DATABASE=" f"{kods_database}" ";" "HOSTNAME=" f"{kods_hostname}" ";" "PORT=" f"{kods_port}" ";" "PROTOCOL=" f"{kods_protocol}" ";" "UID=" f"{kods_username}" ";" "PWD=" f"{kods_password}" ";"
kods_conn = idb.connect(kods_conn_str, "", "")
print("\n\nConnected to Prod TLOG Database.")



Connected to Prod TLOG Database.


In [4]:
#PID DB Connection
dsn = "ibm_db_sa://svcsoat1:l99H1wZf@u060dbc2.kroger.com:50000/DB20TCP"
src_conn = create_engine(dsn)
print("\n\nConnected to PID Database.")



Connected to PID Database.


In [5]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings(action="ignore", category=UserWarning)
warnings.filterwarnings(action="ignore", category=FutureWarning)
warnings.filterwarnings(action="ignore", category=DeprecationWarning)

In [ ]:
##Notes about the code!!!
#1) union_df_sorted - PID FT Query DF
#2) df_Base - Base/Common UPC DF
#3) df_SC - Sales Chanel Query DF
#4) df_SV - Sales Variant Query DF
#5) df_PV - Procurement Query DF
#6) df_snap - SNAP Query DF
#7) df_rbp - RBP Query DF
#8) df_klp - KLP/PDT DF
#9) df_fam - FAM_CD DF
#10) ** PIM_df - PIM QUERY!!
#11) common_upc_list - df to figure out common Items between PID & PIM.

## PIMMART QUERY ##

In [6]:
PIM_Query="""
SELECT DISTINCT
SUBSTRING(A.GTIN_NO,1,13) AS CON_UPC_NO
,D.SV_SV_FL -- SALES_VARIANT
,C.PV_FL -- PV_FLAG
,B.SLS_CHA_CD -- SALES_CHANNEL
,B.PMY_DPT_CD -- PRIMARY
,B.REC_DPT_CD -- RECAP
,B.DPT_CD -- DEPT
,B.[COM_CD] -- COMMODITY
,B.[SUBCOM_CD] --SUBCOMMODITY
--,B.DPT_DSC
,B.SNAP_ELG_FL --SNAP
,RBP_GRP_ID --RBP_GRP_ID
,RBP_GRP_NAM  --RBP_GRP_DESC
,C.KMP_KLP_CD -- KLP_KMP_CD
,C.CPT_BRN_PDT_CLS_CD  --PDT_CLS_CD
,E.FAM_CD
FROM CURIDQ.PIMMART.ITM A
RIGHT JOIN CURIDQ.PIMMART.ITM_XTN_02 B
ON A.ITM_ID=B.ITM_ID
RIGHT JOIN CURIDQ.PIMMART.ITM_XTN_03 C
ON A.ITM_ID=C.ITM_ID
RIGHT JOIN CURIDQ.PIMMART.ITM_XTN_01 D
ON A.ITM_ID=D.ITM_ID
RIGHT JOIN CURIDQ.PIMMART.ITM_EPOS_ATB E
ON A.ITM_ID = E.ITM_ID
ORDER BY CON_UPC_NO;
"""

In [7]:
%%time
# Execute the PIM query and read the data into a DataFrame
PIM_df = pd.read_sql(PIM_Query, src_conn_pim)

PIM_df.head()

CPU times: total: 30.5 s
Wall time: 48.9 s


,CON_UPC_NO,SV_SV_FL,PV_FL,SLS_CHA_CD,PMY_DPT_CD,REC_DPT_CD,DPT_CD,COM_CD,SUBCOM_CD,SNAP_ELG_FL,RBP_GRP_ID,RBP_GRP_NAM,KMP_KLP_CD,CPT_BRN_PDT_CLS_CD,FAM_CD
0,0000000000000,No,None,None,03,87,51,906,90603,None,None,None,None,None,None
1,0000000000001,Yes,None,None,03,03,03,186,69900,None,None,None,None,None,None
2,0000000000002,Yes,Yes,STORE,03,03,04,202,23231,F,None,None,None,None,None
3,0000000000003,Yes,None,None,01,01,45,045,10206,F,None,None,None,None,None
4,0000000000014,Yes,Yes,None,03,03,03,186,69900,None,None,None,None,None,None


## Base UPC ##

In [8]:
Base_Query= """
SELECT CON_UPC_NO FROM PID.PIDORDEN
UNION
SELECT CON_UPC_NO FROM PID.PIDPDTCO
UNION
SELECT CON_UPC_NO FROM PID.PIDDSDIT
UNION
SELECT CON_UPC_NO FROM PID.PIDRMOTE
UNION 
SELECT CAS_UPC_NO AS CON_UPC_NO FROM PID.PIDPDTCA
UNION
SELECT CON_UPC_NO FROM PID.PIDFAMCD
UNION
SELECT (SUBSTR(CON_UPC_NO,1,1)||SUBSTR(CON_UPC_NO,3,1)||SUBSTR(CON_UPC_NO,2,1)||SUBSTR(CON_UPC_NO,4,10)) AS CON_UPC_NO FROM PID.PIDFAMCD
;
"""

In [9]:
%%time
# Execute the Base query and read the data into a DataFrame
df_Base = pd.read_sql(Base_Query, src_conn)

#rename column name
df_Base.rename(columns={'con_upc_no': 'CON_UPC_NO'}, inplace=True)

# Find common UPCs between PIM_df and df_Base -- This is for Sync Items.
common_upcs = pd.merge(PIM_df['CON_UPC_NO'], df_Base['CON_UPC_NO'], on='CON_UPC_NO', how='inner')

df_Base.head()
# Output the list of common UPCs
common_upc_list = common_upcs['CON_UPC_NO'].tolist()
#print(common_upc_list[:15])

# Total number of Items to consider for Sync Item Feed.
#num_records = len(common_upcs)
# Print the number of records
#print("Number of records:", num_records)

CPU times: total: 18.3 s
Wall time: 1min 27s


## PID FT QUERY ##


In [10]:
PID_Query= """
WITH FT AS
(
SELECT TRIM(B.LFO_GRP_CLS_ID || B.LFO_GRP_SUB_ID) AS SUBCOM,   
        B.CPT_COM_CD AS FT_COMM_CODE, 
        COM_TBL.DEPT AS FT_DEPT_CODE,
        REC_TBL.PRI_DEPT AS FT_PRIMARY_DEPT_CODE,
        DPT_TBL.REC_DEPT AS FT_RECAP_DEPT_CODE
FROM PID.PIDSBCOM B
JOIN (SELECT A.KEY_VLU_CD AS COM, SUBSTR(A.GNC_TX,1,30) AS COM_DESC, SUBSTR(A.GNC_TX,32,2) AS DEPT FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0293') 
COM_TBL ON COM_TBL.COM = B.CPT_COM_CD
JOIN (SELECT A.KEY_VLU_CD AS DEPT, SUBSTR(A.GNC_TX,1,15) AS DEPT_DESC, SUBSTR(A.GNC_TX,16,2) AS REC_DEPT FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0295')
DPT_TBL ON COM_TBL.DEPT = DPT_TBL.DEPT
JOIN (SELECT A.KEY_VLU_CD AS REC_DEPT, SUBSTR(A.GNC_TX,1,15) AS RECAP_DESC, SUBSTR(A.GNC_TX,16,2) AS PRI_DEPT FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0344')
REC_TBL ON DPT_TBL.REC_DEPT = REC_TBL.REC_DEPT 
JOIN (SELECT A.KEY_VLU_CD AS PRI_DEPT, SUBSTR(A.GNC_TX, 1, 15) AS PRI_DESC FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0296')
PRI_TBL ON REC_TBL.PRI_DEPT = PRI_TBL.PRI_DEPT
WHERE COM_TBL.COM = '159'

UNION

SELECT DISTINCT COM_TBL.SUBCOM,
       COM_TBL.COM AS FT_COMM_CODE,
       COM_TBL.DEPT AS FT_DEPT_CODE,
       REC_TBL.PRI_DEPT AS FT_PRIMARY_DEPT_CODE,
       DPT_TBL.REC_DEPT AS FT_RECAP_DEPT_CODE
FROM (  SELECT SS.CPT_DPT_CD AS DEPT, SS.CPT_COM_CD AS COM, SS.SUB AS SUBCOM
        FROM (         SELECT CPT_DPT_CD, CPT_COM_CD, CONCAT(LFO_GRP_CLS_ID, LFO_GRP_SUB_ID) AS SUB FROM PID.PIDPDTCA WHERE CPT_COM_CD = '159') 
        SS         GROUP BY SS.CPT_DPT_CD, SS.CPT_COM_CD, SS.SUB
        UNION ALL
        SELECT CPT_DPT_CD AS DEPT, CPT_COM_CD AS COM, TRIM(LFO_GRP_CLS_ID || LFO_GRP_SUB_ID) AS SUBCOM FROM PID.PIDSBCOM WHERE CPT_COM_CD = '159'
     ) COM_TBL
JOIN(SELECT A.KEY_VLU_CD AS DEPT, SUBSTR(A.GNC_TX,1,15) AS DEPT_DESC, SUBSTR(A.GNC_TX,16,2) AS REC_DEPT FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0295'      ) 
DPT_TBL ON COM_TBL.DEPT = DPT_TBL.DEPT
JOIN (SELECT A.KEY_VLU_CD AS REC_DEPT, SUBSTR(A.GNC_TX,1,15) AS RECAP_DESC, SUBSTR(A.GNC_TX,16,2) AS PRI_DEPT FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0344'      )
REC_TBL ON  DPT_TBL.REC_DEPT = REC_TBL.REC_DEPT 
JOIN (SELECT A.KEY_VLU_CD AS PRI_DEPT, SUBSTR(A.GNC_TX,1,15) AS PRI_DESC FROM TEC.TECCODES A WHERE A.TBL_NAM_ID = '0296'      ) 
PRI_TBL ON REC_TBL.PRI_DEPT = PRI_TBL.PRI_DEPT
WHERE COM_TBL.COM = '159'
),
----------------SECOND SUBQUERY FOR PIDPDTCA
PIDPDTCA AS (SELECT CAS_UPC_NO,CPT_DPT_CD,CPT_COM_CD,TRIM(LFO_GRP_CLS_ID || LFO_GRP_SUB_ID) AS SUBCOM FROM PID.PIDPDTCA)
SELECT * FROM (
SELECT DISTINCT 
Z.CON_UPC_NO AS CON_UPC_NO_ACT,
B.CON_UPC_NO AS CON_UPC_NO_TCO,
Z.CAS_UPC_NO AS CAS_UPC_NO_ACT,
E.CAS_UPC_NO AS CAS_UPC_NO_TCA, 
X.FT_PRIMARY_DEPT_CODE AS FT_PRIMARY_DEPT_CODE,
X.FT_RECAP_DEPT_CODE AS FT_RECAP_DEPT_CODE,
X.FT_DEPT_CODE AS FT_DEPT_CODE,
X.FT_COMM_CODE AS FT_COMM_CODE,
E.SUBCOM,
'CASE_NOT_EQUAL_CONSUMER' AS TABLE_NAME
FROM 
( 
SELECT CON_UPC_NO,CAS_UPC_NO, BIL_DIV_NO FROM PID.PIDORDEN
WHERE PID.PIDORDEN.QPS_SCN_CD IN ('W','S','C')
UNION
SELECT CON_UPC_NO, CAS_UPC_NO,BIL_DIV_NO FROM PID.PIDDSDIT
WHERE PID.PIDDSDIT.QPS_SCN_CD IN ('W','S','C')
UNION
SELECT CON_UPC_NO, CAS_UPC_NO, SYS_ID AS BIL_DIV_NO FROM PID.PIDRMOTE
WHERE CON_UPC_NO != CAS_UPC_NO
) Z           
FULL OUTER JOIN PID.PIDPDTCO B ON Z.CON_UPC_NO = B.CON_UPC_NO
FULL OUTER JOIN PIDPDTCA E ON Z.CAS_UPC_NO = E.CAS_UPC_NO
FULL OUTER JOIN FT X ON X.FT_DEPT_CODE = E.CPT_DPT_CD
AND X.FT_COMM_CODE = E.CPT_COM_CD
AND X.SUBCOM = E.SUBCOM
--AND Z.CON_UPC_NO != Z.CAS_UPC_NO   (WRONG)
)CAS_NOT_EQUAL_CON
WHERE FT_PRIMARY_DEPT_CODE IS NOT NULL
AND SUBCOM IS NOT NULL

UNION
------------------------------------------------------------END OF CAS NOT EQUAL CON

-- CAS=CON-----

SELECT * FROM ( 
SELECT DISTINCT Z.CON_UPC_NO AS CON_UPC_NO_ACT,
B.CON_UPC_NO AS CON_UPC_NO_TCO,
Z.CAS_UPC_NO AS CAS_UPC_NO_ACT,
E.CAS_UPC_NO AS CAS_UPC_NO_TCA,
X.FT_PRIMARY_DEPT_CODE AS FT_PRIMARY_DEPT_CODE,
X.FT_RECAP_DEPT_CODE AS FT_RECAP_DEPT_CODE,
X.FT_DEPT_CODE AS FT_DEPT_CODE,
X.FT_COMM_CODE AS FT_COMM_CODE,
E.SUBCOM,
'CASE_EQUAL_CONSUMER' AS TABLE_NAME
FROM (
SELECT CON_UPC_NO,CAS_UPC_NO, BIL_DIV_NO FROM PID.PIDORDEN
WHERE PID.PIDORDEN.QPS_SCN_CD IN ('W','S','C')
UNION
SELECT CON_UPC_NO, CAS_UPC_NO,BIL_DIV_NO FROM PID.PIDDSDIT
WHERE PID.PIDDSDIT.QPS_SCN_CD IN ('W','S','C')
UNION  
SELECT CON_UPC_NO, CAS_UPC_NO, SYS_ID AS BIL_DIV_NO FROM PID.PIDRMOTE
) Z
FULL OUTER JOIN PID.PIDPDTCO B ON Z.CON_UPC_NO = B.CON_UPC_NO
FULL OUTER JOIN PIDPDTCA E ON Z.CON_UPC_NO = E.CAS_UPC_NO
FULL OUTER JOIN FT X ON X.FT_DEPT_CODE=E.CPT_DPT_CD
AND X.FT_COMM_CODE=E.CPT_COM_CD
AND X.SUBCOM=E.SUBCOM
) CAS_EQUAL_CON
WHERE FT_PRIMARY_DEPT_CODE IS NOT NULL
AND SUBCOM IS NOT NULL

----------------------------------------------------------------------------END OF CAS EQUAL CON
UNION

SELECT CON_UPC_NO_ACT,CON_UPC_NO_TCO,CAS_UPC_NO_ACT,'' AS CAS_UPC_NO_TCA ,FT_PRIMARY_DEPT_CODE, FT_RECAP_DEPT_CODE, FT_DEPT_CODE, FT_COMM_CODE,SUBCOM, 'FULL QUERY' AS TABLE_NAME
FROM(
SELECT 
Z.CON_UPC_NO AS CON_UPC_NO_ACT,
B.CON_UPC_NO AS CON_UPC_NO_TCO,
Z.CAS_UPC_NO AS CAS_UPC_NO_ACT,
REC_TBL.PRI_DEPT AS FT_PRIMARY_DEPT_CODE
,DPT_TBL.REC_DEPT AS FT_RECAP_DEPT_CODE
,COM_TBL.DEPT AS FT_DEPT_CODE
,C.CPT_COM_CD AS FT_COMM_CODE
,TRIM(B.LFO_GRP_CLS_ID  || B.LFO_GRP_SUB_ID) AS SUBCOM
        FROM 
        (
          SELECT CON_UPC_NO,CAS_UPC_NO, BIL_DIV_NO FROM PID.PIDORDEN
          WHERE PID.PIDORDEN.QPS_SCN_CD IN ('W','S','C')
          UNION 
          SELECT CON_UPC_NO,CAS_UPC_NO, BIL_DIV_NO
           FROM PID.PIDDSDIT
           WHERE PID.PIDDSDIT.QPS_SCN_CD IN ('W','S','C')
        UNION
          SELECT CON_UPC_NO,CAS_UPC_NO, SYS_ID AS BIL_DIV_NO FROM PID.PIDRMOTE
        ) Z
        FULL OUTER JOIN PID.PIDPDTCO B ON (Z.CON_UPC_NO=B.CON_UPC_NO)
        FULL OUTER JOIN PID.PIDSBCOM C ON (B.LFO_GRP_CLS_ID=C.LFO_GRP_CLS_ID AND B.LFO_GRP_SUB_ID=C.LFO_GRP_SUB_ID)
        FULL OUTER JOIN (SELECT  A.KEY_VLU_CD AS COM,
                SUBSTR(A.GNC_TX,1,30) AS COM_DESC,
                SUBSTR(A.GNC_TX,32,2) AS DEPT
                FROM TEC.TECCODES A
               WHERE A.TBL_NAM_ID = '0293'
                 AND A.KEY_VLU_CD <> '159') COM_TBL
                ON (C.CPT_COM_CD=COM_TBL.COM)
        LEFT OUTER JOIN (SELECT  A.KEY_VLU_CD AS DEPT,
                SUBSTR(A.GNC_TX,1,15) AS DEPT_DESC,
                SUBSTR(A.GNC_TX,16,2) AS REC_DEPT
               FROM TEC.TECCODES A
                WHERE A.TBL_NAM_ID = '0295') DPT_TBL
                ON (COM_TBL.DEPT=DPT_TBL.DEPT)
        LEFT OUTER JOIN (SELECT  A.KEY_VLU_CD AS REC_DEPT,
                SUBSTR(A.GNC_TX,1,15) AS RECAP_DESC,
                SUBSTR(A.GNC_TX,16,2) AS PRI_DEPT
                FROM TEC.TECCODES A
                WHERE A.TBL_NAM_ID = '0344') REC_TBL
                ON (DPT_TBL.REC_DEPT = REC_TBL.REC_DEPT)
        LEFT OUTER JOIN (SELECT  A.KEY_VLU_CD AS PRI_DEPT,
                SUBSTR(A.GNC_TX,1,15) AS PRI_DESC
                FROM TEC.TECCODES A
                WHERE A.TBL_NAM_ID = '0296') PRI_TBL
                ON (REC_TBL.PRI_DEPT = PRI_TBL.PRI_DEPT) 
                AND COM_TBL.COM != '159'                       
        GROUP BY Z.CON_UPC_NO,Z.CAS_UPC_NO,B.CON_UPC_NO, B.LFO_GRP_CLS_ID, B.LFO_GRP_SUB_ID, C.CPT_COM_CD, COM_TBL.COM_DESC, COM_TBL.DEPT, DPT_TBL.DEPT_DESC, DPT_TBL.REC_DEPT
                , REC_TBL.RECAP_DESC, REC_TBL.PRI_DEPT, PRI_TBL.PRI_DESC, C.CPT_COM_DSC_TX, B.CON_DSC_TX, B.CON_SIZ_TX
     --WITH UR
) FULL_QUERY
WHERE FT_COMM_CODE <>'159'

    """

In [11]:
%%time
# Execute the PID query and read the data into a DataFrame
df = pd.read_sql(PID_Query, src_conn)

CPU times: total: 36 s
Wall time: 22min 4s


In [12]:
# Create separate DataFrames for each criteria for UNION
df1 = df[['con_upc_no_act', 'ft_primary_dept_code', 'ft_recap_dept_code', 'ft_dept_code', 'ft_comm_code', 'subcom', 'table_name']]
df2 = df[['con_upc_no_tco', 'ft_primary_dept_code', 'ft_recap_dept_code', 'ft_dept_code', 'ft_comm_code', 'subcom', 'table_name']]
df3 = df[['cas_upc_no_act', 'ft_primary_dept_code', 'ft_recap_dept_code', 'ft_dept_code', 'ft_comm_code', 'subcom', 'table_name']]
df4 = df[['cas_upc_no_tca', 'ft_primary_dept_code', 'ft_recap_dept_code', 'ft_dept_code', 'ft_comm_code', 'subcom', 'table_name']]

#rename column names
df1 = df1.rename(columns={"con_upc_no_act": "CON_UPC_NO"})
df2 = df2.rename(columns={"con_upc_no_tco": "CON_UPC_NO"})
df3 = df3.rename(columns={"cas_upc_no_act": "CON_UPC_NO"})
df4 = df4.rename(columns={"cas_upc_no_tca": "CON_UPC_NO"})

# Concatenate the DataFrames vertically and ignore the original index
union_df = pd.concat([df1, df2, df3, df4], ignore_index=True)

# Remove duplicate rows
union_df = union_df.drop_duplicates()

# Remove rows where 'CON_UPC_NO' is null or blank
union_df = union_df[~union_df['CON_UPC_NO'].isnull() & (union_df['CON_UPC_NO'] != '')]

# Sort the DataFrame by 'CON_UPC_NO' column
union_df_sort = union_df.sort_values(by='CON_UPC_NO')

# Reset the index of the sorted DataFrame
union_df_sort = union_df_sort.reset_index(drop=True)

#below code will resolve issue for 
#0000000000039,Yes,,STORE,01,01,45,045,10199,F,,,,,
#0000000000039,Yes,,STORE,10,49,55,159,97000,F,,,,,

# Filter records where COM_CD = 159
df_159 = union_df_sort[union_df_sort['ft_comm_code'] == '159']

# Filter records where COM_CD != 159
df_not_159 = union_df_sort[union_df_sort['ft_comm_code'] != '159']

# Find the common 'con_upc_no' values
common_upc = set(df_159['CON_UPC_NO']).intersection(df_not_159['CON_UPC_NO'])

# Drop the common 'con_upc_no' records from 'df_159'
df_159 = df_159[~df_159['CON_UPC_NO'].isin(common_upc)]

# Concatenate 'df_159' and 'df_not_159' after dropping common records
final_df = pd.concat([df_159, df_not_159])

# Sort the final dataset if necessary
union_df_sorted = final_df.sort_values(by=['CON_UPC_NO'])

# Reset the index of the final dataset
union_df_sorted = union_df_sorted.reset_index(drop=True)

union_df_sort.head()

,CON_UPC_NO,ft_primary_dept_code,ft_recap_dept_code,ft_dept_code,ft_comm_code,subcom,table_name
0,0000000000001,03,03,03,186,69900,FULL QUERY
1,0000000000002,03,03,04,202,23231,FULL QUERY
2,0000000000003,01,01,45,045,10206,FULL QUERY
3,0000000000004,01,01,45,045,10205,FULL QUERY
4,0000000000005,03,03,66,218,74092,FULL QUERY


## Sales Chanel ##

In [13]:
SC_Query= """
SELECT 
DISTINCT BASE_CON_UPC_NO as CON_UPC_NO
 ,CASE WHEN DIRECT_SHIP_CON_UPC_NO IS NOT NULL AND SHIP_CON_UPC_NO IS NULL AND STORE_UPC IS NULL AND DIRECT_SHIP_SHIP_STORE = '' AND DIRECT_SHIP_SHIP = '' AND SHIP_STORE = '' AND DIRECT_SHIP_STORE = '' AND NONE1 = '' THEN 'DIRECT SHIP'
WHEN SHIP_CON_UPC_NO IS NOT NULL AND DIRECT_SHIP_CON_UPC_NO IS NULL AND STORE_UPC IS NULL AND DIRECT_SHIP_SHIP_STORE = '' AND DIRECT_SHIP_SHIP = '' AND SHIP_STORE = '' AND DIRECT_SHIP_STORE = '' AND NONE1 = '' THEN 'SHIP'
WHEN STORE_UPC IS NOT NULL AND SHIP_CON_UPC_NO IS NULL AND DIRECT_SHIP_CON_UPC_NO IS NULL AND DIRECT_SHIP_SHIP_STORE = '' AND DIRECT_SHIP_SHIP = '' AND SHIP_STORE = '' AND DIRECT_SHIP_STORE = '' AND NONE1 = '' THEN 'STORE'
WHEN DIRECT_SHIP_CON_UPC_NO IS NOT NULL AND SHIP_CON_UPC_NO IS NOT NULL AND STORE_UPC IS NOT NULL AND DIRECT_SHIP_SHIP_STORE IS NOT NULL 
AND DIRECT_SHIP_SHIP = '' AND SHIP_STORE = '' AND DIRECT_SHIP_STORE = '' AND NONE1 = '' THEN 'DIRECT SHIP; SHIP; STORE'
WHEN DIRECT_SHIP_CON_UPC_NO IS NOT NULL AND SHIP_CON_UPC_NO IS NOT NULL AND STORE_UPC IS NULL AND DIRECT_SHIP_SHIP_STORE = ''
AND DIRECT_SHIP_SHIP IS NOT NULL AND SHIP_STORE = '' AND DIRECT_SHIP_STORE = '' AND NONE1 = '' THEN 'DIRECT SHIP; SHIP'
WHEN DIRECT_SHIP_CON_UPC_NO IS NULL AND SHIP_CON_UPC_NO IS NOT NULL AND STORE_UPC IS NOT NULL AND DIRECT_SHIP_SHIP_STORE = ''
AND DIRECT_SHIP_SHIP = '' AND SHIP_STORE IS NOT NULL AND DIRECT_SHIP_STORE = '' AND NONE1 = '' THEN 'SHIP; STORE'
WHEN DIRECT_SHIP_CON_UPC_NO IS NOT NULL AND SHIP_CON_UPC_NO IS NULL AND STORE_UPC IS NOT NULL AND DIRECT_SHIP_SHIP_STORE = ''
AND DIRECT_SHIP_SHIP = '' AND SHIP_STORE = '' AND DIRECT_SHIP_STORE IS NOT NULL AND NONE1 = '' THEN 'DIRECT SHIP; STORE'
WHEN DIRECT_SHIP_CON_UPC_NO IS NULL AND SHIP_CON_UPC_NO IS NULL AND STORE_UPC IS NULL AND DIRECT_SHIP_SHIP_STORE = '' AND
        DIRECT_SHIP_SHIP = '' AND SHIP_STORE = '' AND DIRECT_SHIP_STORE = '' AND NONE1 IS NOT NULL THEN 'NONE'
ELSE 'NOTHING' END AS SALES_CHNL_AS 

FROM 
(SELECT
    DISTINCT PDTCO.CON_UPC_NO AS BASE_CON_UPC_NO
   ,CASE WHEN PIDORDEN_DSHIP.CON_UPC_NO IS NOT NULL THEN 'DIRECT SHIP' ELSE PIDORDEN_DSHIP.CON_UPC_NO END AS DIRECT_SHIP_CON_UPC_NO
   ,CASE WHEN PIDORDEN_SHIP.CON_UPC_NO IS NOT NULL THEN 'SHIP' ELSE PIDORDEN_SHIP.CON_UPC_NO END AS SHIP_CON_UPC_NO
   ,CASE WHEN PIDORDEN_STORE1.CON_UPC_NO IS NOT NULL OR PIDDSDIT_STORE2.CON_UPC_NO IS NOT NULL OR PIDRMOTE_STORE3.CON_UPC_NO IS NOT NULL THEN 'STORE' ELSE PIDORDEN_STORE1.CON_UPC_NO END AS STORE_UPC 
   ,CASE WHEN PIDORDEN_DSHIP.CON_UPC_NO IS NOT NULL AND PIDORDEN_SHIP.CON_UPC_NO IS NOT NULL AND (PIDORDEN_STORE1.CON_UPC_NO IS NOT NULL OR PIDDSDIT_STORE2.CON_UPC_NO IS NOT NULL OR PIDRMOTE_STORE3.CON_UPC_NO IS NOT NULL)
        THEN 'DIRECT SHIP; SHIP;STORE' ELSE '' END AS DIRECT_SHIP_SHIP_STORE
   ,CASE WHEN PIDORDEN_DSHIP.CON_UPC_NO IS NOT NULL AND PIDORDEN_SHIP.CON_UPC_NO IS NOT NULL AND (PIDORDEN_STORE1.CON_UPC_NO IS NULL AND PIDDSDIT_STORE2.CON_UPC_NO IS NULL AND PIDRMOTE_STORE3.CON_UPC_NO IS NULL) 
THEN 'DIRECT SHIP; SHIP' ELSE '' END AS DIRECT_SHIP_SHIP
   ,CASE WHEN PIDORDEN_SHIP.CON_UPC_NO IS NOT NULL AND (PIDORDEN_STORE1.CON_UPC_NO IS NOT NULL OR PIDDSDIT_STORE2.CON_UPC_NO IS NOT NULL OR PIDRMOTE_STORE3.CON_UPC_NO IS NOT NULL) AND PIDORDEN_DSHIP.CON_UPC_NO IS NULL
THEN 'SHIP; STORE' ELSE '' END AS SHIP_STORE
   ,CASE WHEN PIDORDEN_DSHIP.CON_UPC_NO IS NOT NULL AND (PIDORDEN_STORE1.CON_UPC_NO IS NOT NULL OR PIDDSDIT_STORE2.CON_UPC_NO IS NOT NULL OR PIDRMOTE_STORE3.CON_UPC_NO IS NOT NULL) AND PIDORDEN_SHIP.CON_UPC_NO IS NULL
THEN 'DIRECT SHIP; STORE' ELSE '' END AS DIRECT_SHIP_STORE
   ,CASE WHEN PIDORDEN_DSHIP.CON_UPC_NO IS NULL AND PIDORDEN_SHIP.CON_UPC_NO IS NULL AND (PIDORDEN_STORE1.CON_UPC_NO IS NULL AND PIDDSDIT_STORE2.CON_UPC_NO IS NULL AND PIDRMOTE_STORE3.CON_UPC_NO IS NULL)
        THEN 'NONE' ELSE '' END AS NONE1
           ,LFO_GRP_CLS_ID,
       LFO_GRP_SUB_ID
   
FROM
    PID.PIDPDTCO AS PDTCO
LEFT OUTER JOIN
    (
    SELECT
         DISTINCT CON_UPC_NO
        ,CAS_UPC_NO
    FROM
        PID.PIDORDEN
    WHERE
        1=1
    AND    BIL_DIV_NO IN ('223')
    AND    (CAT_ID = '02' OR CAT_ID = '05')
    AND    DIV_CAS_STU_CD = 'A'
    AND    QPS_SCN_CD in ('W','S','C')
    ) AS PIDORDEN_DSHIP
ON
    PDTCO.CON_UPC_NO = PIDORDEN_DSHIP.CON_UPC_NO
LEFT OUTER JOIN
    (
    SELECT
         DISTINCT CON_UPC_NO
        ,CAS_UPC_NO
    FROM
        PID.PIDORDEN
    WHERE
        1=1
    AND    BIL_DIV_NO IN ('223')
    AND    (CAT_ID = '01' OR CAT_ID = '03')
    AND    DIV_CAS_STU_CD = 'A'
    AND    QPS_SCN_CD in ('W','S','C')
    ) AS PIDORDEN_SHIP
ON
    PDTCO.CON_UPC_NO = PIDORDEN_SHIP.CON_UPC_NO
LEFT OUTER JOIN
    (
    SELECT
         DISTINCT CON_UPC_NO
        ,CAS_UPC_NO
    FROM
        PID.PIDORDEN
    WHERE
        1=1
    AND    BIL_DIV_NO in ('011','014','016','018','021','023','024','026','029','034','035', '090','223','531','533','534','540','615','620','660','700','701','704','703','705','706','708','097')
    AND    DIV_CAS_STU_CD = 'A'
    AND    QPS_SCN_CD in ('W','S','C')                    
    ) AS PIDORDEN_STORE1
ON
    PDTCO.CON_UPC_NO = PIDORDEN_STORE1.CON_UPC_NO
LEFT OUTER JOIN
    (
    SELECT
         DISTINCT CON_UPC_NO
        ,CAS_UPC_NO
    FROM
        PID.PIDDSDIT
    WHERE
        1=1
    AND    BIL_DIV_NO in ('011','014','016','018','021','023','024','025','026','029','034','223','035','090','531','540','533','534','615','620','660','700','701','704','703','705','706','708','097')
    AND    BIL_STU_CD = '01'
    AND    QPS_SCN_CD in ('W','S','C')
    ) AS PIDDSDIT_STORE2
ON
    PDTCO.CON_UPC_NO = PIDDSDIT_STORE2.CON_UPC_NO
LEFT OUTER JOIN
    (
    SELECT
         DISTINCT CON_UPC_NO
        , CAS_UPC_NO
    FROM
        PID.PIDRMOTE
    WHERE
        SYS_ID = 'MAG'
        AND    STU_CD = 'A'
    ) AS PIDRMOTE_STORE3
ON
    PDTCO.CON_UPC_NO = PIDRMOTE_STORE3.CON_UPC_NO) ABC 
"""

In [14]:
%%time
# Execute the SC query and read the data into a DataFrame
df_SC = pd.read_sql(SC_Query, src_conn)

#rename uppercase names
df_SC = df_SC.rename(columns={"con_upc_no": "CON_UPC_NO"})

df_SC.head()

CPU times: total: 12.8 s
Wall time: 1min 22s


,CON_UPC_NO,sales_chnl_as
0,0000000000001,NONE
1,0000000000002,STORE
2,0000000000003,NONE
3,0000000000004,NONE
4,0000000000005,NONE


## SV Query ##

In [15]:
query_cons = """
SELECT DISTINCT A.CON_UPC_NO   
FROM PID.PIDPDTCO A
 INNER JOIN 
 (
SELECT CON_UPC_NO,CAS_UPC_NO
         FROM PID.PIDORDEN
         WHERE DIV_CAS_STU_CD IN ('A','I','N')
         AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','026','029','034','035', '090','223','531','533','534','540','615','620','660','700','701','704','703','705','706','708','097')
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDDSDIT
          WHERE BIL_STU_CD = '01'
          AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','025','026','029','034','223','035','090','531','540','533','534','615','620','660','700','701','704','703','705','706','708','097')     
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDRMOTE
          WHERE STU_CD = 'A'   
          ) ABC ON ABC.CON_UPC_NO=A.CON_UPC_NO
LEFT JOIN PID.PIDPDTCA B ON ABC.CON_UPC_NO=B.CAS_UPC_NO AND CAS_SHP_FL='';
"""       

In [16]:
%%time
# Execute the SV query and read the data into a DataFrame
df_sv_cons = pd.read_sql(query_cons, src_conn)

#rename uppercase names
df_sv_cons = df_sv_cons.rename(columns={"con_upc_no": "CON_UPC_NO"})

df_sv_cons.head()

CPU times: total: 6.05 s
Wall time: 40.2 s


,CON_UPC_NO
0,0000000000002
1,0000000000014
2,0000000000015
3,0000000000019
4,0000000000023


In [17]:
query_case = """
SELECT DISTINCT ABC.Cas_UPC_NO  
FROM PID.PIDPDTCO A
 INNER JOIN 
 (
SELECT CON_UPC_NO,CAS_UPC_NO
         FROM PID.PIDORDEN
         WHERE DIV_CAS_STU_CD IN ('A','I','N')
         AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','026','029','034','035', '090','223','531','533','534','540','615','620','660','700','701','704','703','705','706','708','097')
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDDSDIT
          WHERE BIL_STU_CD = '01'
          AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','025','026','029','034','223','035','090','531','540','533','534','615','620','660','700','701','704','703','705','706','708','097')     
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDRMOTE
          WHERE STU_CD = 'A'
          ) ABC ON ABC.CON_UPC_NO=A.CON_UPC_NO
LEFT JOIN PID.PIDPDTCA B ON ABC.CON_UPC_NO=B.CAS_UPC_NO AND CAS_SHP_FL='';
"""    

In [18]:
# Execute the SV query and read the data into a DataFrame
df_sv_case = pd.read_sql(query_case, src_conn)

#rename uppercase names
df_sv_case = df_sv_case.rename(columns={"cas_upc_no": "CON_UPC_NO"})

df_sv_case.head()

,CON_UPC_NO
0,0000000000002
1,0000000000014
2,0000000000015
3,0000000000040
4,0000000000059


In [19]:
# Merge the two dataframes on UPC using outer join
df_SV = pd.merge(df_sv_cons, df_sv_case, on='CON_UPC_NO', how='outer', indicator=True)

# Create a new column 'SV' based on the indicator
df_SV['SV_FL'] = df_SV['_merge'].map({'both': 'Yes', 'left_only': 'Yes', 'right_only': 'No'})

# Drop the unnecessary columns and rename the column
df_SV.drop(['_merge'], axis=1, inplace=True)

# Sort the dataframe by UPC
df_SV.sort_values(by='CON_UPC_NO', inplace=True)

# Reset the index of the final dataframe
df_SV.reset_index(drop=True, inplace=True)

# Print the final dataframe
df_SV.head()

,CON_UPC_NO,SV_FL
0,0000000000002,Yes
1,0000000000014,Yes
2,0000000000015,Yes
3,0000000000019,Yes
4,0000000000023,Yes


In [20]:
query_sv_shipper = """
select DISTINCT a.con_upc_no AS UPC, CAS_SHP_FL as SHIPPER_ID  from pid.pidpdtco a
inner join 
 (
 SELECT CON_UPC_NO,CAS_UPC_NO
         FROM PID.PIDORDEN
         WHERE DIV_CAS_STU_CD IN ('A','I','N')
         AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','026','029','034','035', '090','223','531','533','534','540','615','620','660','700','701','704','703','705','706','708','097')
        UNION
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDDSDIT
          WHERE BIL_STU_CD = '01'
          AND    QPS_SCN_CD in ('W','S','C')
            AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','025','026','029','034','223','035','090','531','540','533','534','615','620','660','700','701','704','703','705','706','708','097')
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDRMOTE
          WHERE STU_CD = 'A'   
) abc
on abc.con_upc_no=a.con_upc_no
inner join pid.pidpdtca b
on abc.cas_upc_no=b.cas_upc_no
and CAS_SHP_FL ='Y'
"""

In [21]:
# Execute the SV query and read the data into a DataFrame
df_sv_shipper = pd.read_sql(query_sv_shipper, src_conn)

#rename uppercase names
df_sv_shipper = df_sv_shipper.rename(columns={"upc": "CON_UPC_NO"})

df_sv_shipper.head()

,CON_UPC_NO,shipper_id
0,0000071651285,Y
1,0000071657833,Y
2,0000071660115,Y
3,0000071660399,Y
4,0000071660494,Y


In [22]:
# Update the 'SV' column based on matching and existing values in df_sv_shipper
matching_upcs = df_sv_shipper[df_sv_shipper['CON_UPC_NO'].isin(df_SV['CON_UPC_NO'])]['CON_UPC_NO']
df_SV.loc[df_SV['CON_UPC_NO'].isin(matching_upcs) & (df_SV['SV_FL'] != 'No'), 'SV_FL'] = 'No'

# Append the non-matching UPCs from df_shipper to df_SV
non_matching_upcs = df_sv_shipper[~df_sv_shipper['CON_UPC_NO'].isin(df_SV['CON_UPC_NO'])]
non_matching_upcs['SV_FL'] = 'No'
df_SV = pd.concat([df_SV, non_matching_upcs], ignore_index=True)

In [23]:
# Reset the index of the final dataframe
df_SV.reset_index(drop=True, inplace=True)

# Print the final dataframe
df_SV.head()

,CON_UPC_NO,SV_FL,shipper_id
0,0000000000002,Yes,NaN
1,0000000000014,Yes,NaN
2,0000000000015,Yes,NaN
3,0000000000019,Yes,NaN
4,0000000000023,Yes,NaN


## PV Query ##

In [24]:
query_conspv = """
SELECT DISTINCT A.CON_UPC_NO   
FROM PID.PIDPDTCO A
 INNER JOIN 
 (
SELECT CON_UPC_NO,CAS_UPC_NO
         FROM PID.PIDORDEN
         WHERE DIV_CAS_STU_CD IN ('A','I','N')
         AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','026','029','034','035', '090','223','531','533','534','540','615','620','660','700','701','704','703','705','706','708','097')
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDDSDIT
          WHERE BIL_STU_CD = '01'
          AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','025','026','029','034','223','035','090','531','540','533','534','615','620','660','700','701','704','703','705','706','708','097')     
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDRMOTE
          WHERE STU_CD = 'A' 
          ) ABC ON ABC.CON_UPC_NO=A.CON_UPC_NO
INNER JOIN PID.PIDPDTCA B ON ABC.CAS_UPC_NO=B.CAS_UPC_NO;
"""       

In [25]:
# Execute the SV query and read the data into a DataFrame
df_pv_cons = pd.read_sql(query_conspv, src_conn)

#rename uppercase names
df_pv_cons = df_pv_cons.rename(columns={"con_upc_no": "CON_UPC_NO"})

df_pv_cons.head()

,CON_UPC_NO
0,0000000000002
1,0000000000014
2,0000000000015
3,0000000000019
4,0000000000023


In [26]:
query_casepv = """
SELECT DISTINCT ABC.Cas_UPC_NO  
FROM PID.PIDPDTCO A
 INNER JOIN 
 (
SELECT CON_UPC_NO,CAS_UPC_NO
         FROM PID.PIDORDEN
         WHERE DIV_CAS_STU_CD IN ('A','I','N')
         AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','026','029','034','035', '090','223','531','533','534','540','615','620','660','700','701','704','703','705','706','708','097')
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDDSDIT
          WHERE BIL_STU_CD = '01'
          AND    QPS_SCN_CD in ('W','S','C')
           AND BIL_DIV_NO IN ('011','014','016','018','021','023','024','025','026','029','034','223','035','090','531','540','533','534','615','620','660','700','701','704','703','705','706','708','097')     
        UNION 
          SELECT CON_UPC_NO, CAS_UPC_NO
          FROM PID.PIDRMOTE
          WHERE STU_CD = 'A'  
          ) ABC ON ABC.CON_UPC_NO=A.CON_UPC_NO
INNER JOIN PID.PIDPDTCA B ON ABC.CAS_UPC_NO=B.CAS_UPC_NO;
"""    

In [27]:
# Execute the SV query and read the data into a DataFrame
df_pv_case = pd.read_sql(query_casepv, src_conn)

#rename uppercase names
df_pv_case = df_pv_case.rename(columns={"cas_upc_no": "CON_UPC_NO"})

# Merge the two dataframes on UPC using outer join
df_PV = pd.merge(df_pv_cons, df_pv_case, on='CON_UPC_NO', how='outer', indicator=True)

# Create a new column 'SV' based on the indicator
df_PV['PV_FL'] = df_PV['_merge'].map({'both': 'Yes', 'left_only': 'No', 'right_only': 'Yes'})

# Drop the unnecessary columns and rename the column
df_PV.drop(['_merge'], axis=1, inplace=True)

# Sort the dataframe by UPC
df_PV.sort_values(by='CON_UPC_NO', inplace=True)

# Reset the index of the final dataframe
df_PV.reset_index(drop=True, inplace=True)

# Print the final dataframe
df_PV.head()

,CON_UPC_NO,PV_FL
0,0000000000002,Yes
1,0000000000014,Yes
2,0000000000015,Yes
3,0000000000019,No
4,0000000000023,No


## SNAP Query ##

In [28]:
Snap_Query= """
SELECT DISTINCT A.CAS_UPC_NO AS CON_UPC_NO
,CASE WHEN (A.FDS_ELG_FL = 'F') THEN 'F' WHEN ( A.FDS_ELG_FL ='' )THEN ''  END AS SNAP_FL
FROM PID.PIDPDTCA A 
INNER JOIN PID.PIDSBCOM B ON A.LFO_GRP_CLS_ID = B.LFO_GRP_CLS_ID AND A.LFO_GRP_SUB_ID  = B.LFO_GRP_SUB_ID
INNER JOIN PID.PIDCASCO C ON A.CAS_UPC_NO = C.CAS_UPC_NO
where c.con_typ_cd in ('1','2')
GROUP BY
A.CAS_UPC_NO, A.FDS_ELG_FL
UNION
SELECT DISTINCT C.CON_UPC_NO
,CASE WHEN (A.FDS_ELG_FL = 'F') THEN 'F' WHEN ( A.FDS_ELG_FL ='' )THEN ''  END AS SNAP_FL
FROM PID.PIDPDTCA A 
INNER JOIN PID.PIDSBCOM B ON A.LFO_GRP_CLS_ID = B.LFO_GRP_CLS_ID AND A.LFO_GRP_SUB_ID  = B.LFO_GRP_SUB_ID
INNER JOIN PID.PIDCASCO C ON A.CAS_UPC_NO = C.CAS_UPC_NO
where c.con_typ_cd in ('1','2')
GROUP BY
C.CON_UPC_NO, A.FDS_ELG_FL;
""" 

In [29]:
%%time
# Execute the SNAP QUery and read the data into a DataFrame
df_snapfl = pd.read_sql(Snap_Query, src_conn)

#rename uppercase names
df_snapfl = df_snapfl.rename(columns={"con_upc_no": "CON_UPC_NO"})

#df_snapfl.head()

# Group by CON_UPC_NO and filter for groups with more than one entry
grouped = df_snapfl.groupby('CON_UPC_NO').filter(lambda x: len(x) > 1)

# Create a single list of unique UPCs
upc_list = grouped['CON_UPC_NO'].unique().tolist()

# Filter the main DataFrame to exclude the UPCs in upc_list - MAIN***
#we are doing this to make 2 datasets, One with single records, and another with 2 records.
df_remaining = df_snapfl[~df_snapfl['CON_UPC_NO'].isin(upc_list)]

#df_remaining.head()
#rename uppercase names
#df_remaining = df_remaining.rename(columns={"con_upc_no": "CON_UPC_NO"})

# Convert the upc_list to a comma-separated string
upc_string = "', '".join(upc_list)

CPU times: total: 40.6 s
Wall time: 4min 3s


In [30]:
# SQL query for remaining UPCs
query5 = """
SELECT B.CON_UPC_NO, A.FDS_ELG_FL as SNAP_FL
FROM PID.PIDSBCOM A
INNER JOIN PID.PIDPDTCO B ON A.LFO_GRP_CLS_ID = B.LFO_GRP_CLS_ID AND A.LFO_GRP_SUB_ID = B.LFO_GRP_SUB_ID
WHERE B.CON_UPC_NO IN ('{upc_list}')
""".format(upc_list=upc_string)

In [31]:
# Execute the SNAP Qery for multiple records and read the data into a DataFrame
df_results2 = pd.read_sql(query5, src_conn)

#rename uppercase names
df_results2 = df_results2.rename(columns={"con_upc_no": "CON_UPC_NO"})

df_results2.head()

# Concatenate the two DataFrames vertically
union_df = pd.concat([df_results2, df_remaining])

# Sort the union_df DataFrame based on a specific column
df_snap = union_df.sort_values('CON_UPC_NO')

df_snap.head()

# Write the DataFrame to CSV
#df_snap.to_csv("C:\\Users\\AS81970\\Desktop\\snapout.csv", index=False)

,CON_UPC_NO,snap_fl
0,0000000000001,
1,0000000000002,F
2,0000000000003,F
3,0000000000014,
4,0000000000015,


## RBP - ID and Desc ##

In [32]:
RBP_Query=""" 
--RBP_GRP_ID, RBP_GRP_DESC
SELECT DISTINCT A.BAS_CON_UPC_NO as con_upc_no,
CASE WHEN B.RBP_CPT_PRC_GRP_ID < 1 THEN '' ELSE CONCAT('GC', LPAD(B.RBP_CPT_PRC_GRP_ID, 6, '0')) END AS RBP_CPT_PRC_GRP_ID,
CASE WHEN B.CPT_PRC_GRP_DSC_TX = 'DESC UNKNOWN' THEN '' ELSE B.CPT_PRC_GRP_DSC_TX END AS CPT_PRC_GRP_DSC_TX
FROM KODS.NRM_BAS_UPC A
JOIN KODS.NRM_CPT_PRC_GRP B
ON A.CPT_PRC_GRP_ID=B.CPT_PRC_GRP_ID
UNION
SELECT DISTINCT A.BAS_CON_UPC_NO as con_upc_no,
CASE WHEN B.RBP_CPT_PRC_GRP_ID < 1 THEN '' ELSE CONCAT('GC', LPAD(B.RBP_CPT_PRC_GRP_ID, 6, '0')) END AS RBP_CPT_PRC_GRP_ID,
CASE WHEN B.CPT_PRC_GRP_DSC_TX = 'DESC UNKNOWN' THEN '' ELSE B.CPT_PRC_GRP_DSC_TX END AS CPT_PRC_GRP_DSC_TX
FROM KODS.NRM_BAS_CPT_PRC_GRP A
JOIN KODS.NRM_CPT_PRC_GRP B
ON A.RBP_CPT_PRC_GRP_ID=B.RBP_CPT_PRC_GRP_ID;
""" 

In [35]:
%%time
# Execute the RBP QUery and read the data into a DataFrame
df_rbp = pd.read_sql(RBP_Query, kods_conn)

df_rbp.head()

CPU times: total: 6.41 s
Wall time: 39 s


,CON_UPC_NO,RBP_CPT_PRC_GRP_ID,CPT_PRC_GRP_DSC_TX
0,0,,
1,0000000000002,,
2,0000000000014,,
3,0000000000015,,
4,0000000000019,,


## KLP/KMP & PDT_CLS Code ##

In [36]:
klp_Query=""" 
SELECT DISTINCT PID.PIDCASCO.CON_UPC_NO,                
TRIM(KLP_KMP_CD) AS KLP_KMP_CD, trim(PDT_CLS_CD) AS PDT_CLS_CD
FROM PID.PIDPDTCA
INNER JOIN PID.PIDCASCO
ON PID.PIDCASCO.CAS_UPC_NO=PID.PIDPDTCA.CAS_UPC_NO
WHERE CON_TYP_CD IN('1','2');
""" 

In [37]:
# Execute the KLP/PDT QUery and read the data into a DataFrame
df_klp = pd.read_sql(klp_Query, src_conn)

#rename uppercase names
df_klp = df_klp.rename(columns={"con_upc_no": "CON_UPC_NO"})

df_klp.head()

,CON_UPC_NO,klp_kmp_cd,pdt_cls_cd
0,0000000000001,,
1,0000000000002,,
2,0000000000003,,
3,0000000000014,,
4,0000000000015,,


## FAM Code ##

In [38]:
fam_Query="""
SELECT DISTINCT CON_UPC_NO,trim(FAM_CD) as FAM_CD
FROM PID.PIDFAMCD 
UNION
SELECT SUBSTR(CON_UPC_NO,1,1)||SUBSTR(CON_UPC_NO,3,1)||SUBSTR(CON_UPC_NO,2,1)||SUBSTR(CON_UPC_NO,4,10) AS CON_UPC_NO ,trim(FAM_CD) as FAM_CD
FROM PID.PIDFAMCD;
"""

In [39]:
%%time
# Execute the KLP/PDT QUery and read the data into a DataFrame
df_fam_cd = pd.read_sql(fam_Query, src_conn)

#rename uppercase names
df_fam_cd = df_fam_cd.rename(columns={"con_upc_no": "CON_UPC_NO"})

# Concatenate the FAM_CD values for each CON_UPC_NO
df_fam = df_fam_cd.groupby('CON_UPC_NO')['fam_cd'].apply(lambda x: '; '.join(x)).reset_index()

df_fam.head()

CPU times: total: 3.94 s
Wall time: 8.93 s


,CON_UPC_NO,fam_cd
0,0000030000017,111
1,0000030000030,111
2,0000030000041,111
3,0000030000047,111
4,0000030000068,111


## Merge all Data on UPC ##

In [40]:
# Merge DataFrames one by one
merged_df = pd.merge(union_df_sorted, df_SC, on='CON_UPC_NO', how='outer')

merged_df = pd.merge(merged_df, df_SV, on='CON_UPC_NO', how='outer')

merged_df = pd.merge(merged_df, df_PV, on='CON_UPC_NO', how='outer')

merged_df = pd.merge(merged_df, df_snap, on='CON_UPC_NO', how='outer')

merged_df = pd.merge(merged_df, df_rbp, on='CON_UPC_NO', how='outer')

merged_df = pd.merge(merged_df, df_klp, on='CON_UPC_NO', how='outer')

merged_df = pd.merge(merged_df, df_fam, on='CON_UPC_NO', how='outer')

merged_df.head()

,CON_UPC_NO,ft_primary_dept_code,ft_recap_dept_code,ft_dept_code,ft_comm_code,subcom,table_name,sales_chnl_as,SV_FL,shipper_id,PV_FL,snap_fl,RBP_CPT_PRC_GRP_ID,CPT_PRC_GRP_DSC_TX,klp_kmp_cd,pdt_cls_cd,fam_cd
0,0000000000001,03,03,03,186,69900,FULL QUERY,NONE,NaN,NaN,NaN,,NaN,NaN,,,NaN
1,0000000000002,03,03,04,202,23231,FULL QUERY,STORE,Yes,NaN,Yes,F,,,,,NaN
2,0000000000003,01,01,45,045,10206,FULL QUERY,NONE,NaN,NaN,NaN,F,NaN,NaN,,,NaN
3,0000000000004,01,01,45,045,10205,FULL QUERY,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000005,03,03,66,218,74092,FULL QUERY,NONE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filter Merged PID Data on Common UPC List ##

In [41]:
# Filter merged_df based on the common list of UPCs
pidfinal_df = merged_df[merged_df['CON_UPC_NO'].isin(common_upc_list)]

In [42]:
# Reorder columns
column_order = ['CON_UPC_NO', 'SV_FL', 'PV_FL', 'sales_chnl_as', 'ft_primary_dept_code','ft_recap_dept_code','ft_dept_code','ft_comm_code','subcom','snap_fl','RBP_CPT_PRC_GRP_ID','CPT_PRC_GRP_DSC_TX','klp_kmp_cd','pdt_cls_cd','fam_cd'] 

pidfinal_df = pidfinal_df[column_order]

In [43]:
# Remove duplicates from pidfinal_df
pidfinal_df = pidfinal_df.drop_duplicates()

## Filter PIM Data on common UPC List ##

In [44]:
# Filter PIM_df based on the common list of UPCs
pimfinal_df = PIM_df[PIM_df['CON_UPC_NO'].isin(common_upc_list)]

In [45]:
# THis piece is added for TEST Env. because of duplicates.
pimfinal_df.drop_duplicates(subset=['CON_UPC_NO'], keep='first', inplace=True)

## Modify PID Final File column names for Comparisson ##

In [46]:
new_column_names = {
    'SV_FL': 'SV_SV_FL',
    'PV_FL': 'PV_FL',
    'sales_chnl_as': 'SLS_CHA_CD',
    'ft_primary_dept_code': 'PMY_DPT_CD',
    'ft_recap_dept_code': 'REC_DPT_CD',
    'ft_dept_code': 'DPT_CD',
    'ft_comm_code': 'COM_CD',
    'subcom': 'SUBCOM_CD',
    'snap_fl': 'SNAP_ELG_FL',
    'RBP_CPT_PRC_GRP_ID': 'RBP_GRP_ID',
    'CPT_PRC_GRP_DSC_TX': 'RBP_GRP_NAM',
    'klp_kmp_cd': 'KMP_KLP_CD',
    'pdt_cls_cd': 'CPT_BRN_PDT_CLS_CD',
    'fam_cd': 'FAM_CD'
}

In [47]:
pidfinal_df = pidfinal_df.rename(columns=new_column_names)

In [48]:
# View column names of pimfinal_df
print(pimfinal_df.columns)

Index(['CON_UPC_NO', 'SV_SV_FL', 'PV_FL', 'SLS_CHA_CD', 'PMY_DPT_CD',
       'REC_DPT_CD', 'DPT_CD', 'COM_CD', 'SUBCOM_CD', 'SNAP_ELG_FL',
       'RBP_GRP_ID', 'RBP_GRP_NAM', 'KMP_KLP_CD', 'CPT_BRN_PDT_CLS_CD',
       'FAM_CD'],
      dtype='object')


In [49]:
# View column names of pidfinal_df
print(pidfinal_df.columns)

Index(['CON_UPC_NO', 'SV_SV_FL', 'PV_FL', 'SLS_CHA_CD', 'PMY_DPT_CD',
       'REC_DPT_CD', 'DPT_CD', 'COM_CD', 'SUBCOM_CD', 'SNAP_ELG_FL',
       'RBP_GRP_ID', 'RBP_GRP_NAM', 'KMP_KLP_CD', 'CPT_BRN_PDT_CLS_CD',
       'FAM_CD'],
      dtype='object')


## Compare 'pimfinal_df' & 'pidfinal_df' and get data for only MISMATCHES ##

In [50]:
# Sort pimfinal_df inplace based on 'CON_UPC_NO' column
pimfinal_df.sort_values(by='CON_UPC_NO', inplace=True)

# Sort pidfinal_df inplace based on 'CON_UPC_NO' column
pidfinal_df.sort_values(by='CON_UPC_NO', inplace=True)

pimfinal_df.replace(np.nan, '', regex=True,inplace=True)
pidfinal_df.replace(np.nan, '', regex=True,inplace=True)

# Merge the dataframes on 'CON_UPC_NO'
finalmerged_df = pidfinal_df.merge(pimfinal_df, on='CON_UPC_NO', suffixes=('_x', '_y'), how='outer')

# Define the columns of interest for comparison
columns_of_interest = ['SV_SV_FL', 'PV_FL', 'SLS_CHA_CD', 'PMY_DPT_CD', 'REC_DPT_CD', 'DPT_CD', 'COM_CD', 'SUBCOM_CD', 'SNAP_ELG_FL', 'RBP_GRP_ID', 'RBP_GRP_NAM', 'KMP_KLP_CD', 'CPT_BRN_PDT_CLS_CD', 'FAM_CD']

# Initialize a list to store mismatched CON_UPC_NO values
mismatched_con_upc = []

# Iterate over the columns of interest
for column in columns_of_interest:
    mask = (finalmerged_df[column + '_x'] != finalmerged_df[column + '_y'])
    mismatched_con_upc.extend(finalmerged_df.loc[mask, 'CON_UPC_NO'].tolist())
    
# Get unique mismatched CON_UPC_NO values
mismatched_con_upc = set(mismatched_con_upc)    

In [51]:
# Create new dataframes with only PID mismatched records
pidfinal_new_df = pidfinal_df[pidfinal_df['CON_UPC_NO'].isin(mismatched_con_upc)]

In [52]:
# Create new dataframes with only PIM mismatched records
pimfinal_new_df = pimfinal_df[pimfinal_df['CON_UPC_NO'].isin(mismatched_con_upc)]

In [53]:
new_column_names_pim = {
    'SV_SV_FL': 'SV_SV_FL_pim',
    'PV_FL': 'PV_FL_pim',
    'SLS_CHA_CD': 'SLS_CHA_CD_pim',
    'PMY_DPT_CD': 'PMY_DPT_CD_pim',
    'REC_DPT_CD': 'REC_DPT_CD_pim',
    'DPT_CD': 'DPT_CD_pim',
    'COM_CD': 'COM_CD_pim',
    'SUBCOM_CD': 'SUBCOM_CD_pim',
    'SNAP_ELG_FL': 'SNAP_ELG_FL_pim',
    'RBP_GRP_ID': 'RBP_GRP_ID_pim',
    'RBP_GRP_NAM': 'RBP_GRP_NAM_pim',
    'KMP_KLP_CD': 'KMP_KLP_CD_pim',
    'CPT_BRN_PDT_CLS_CD': 'CPT_BRN_PDT_CLS_CD_pim',
    'FAM_CD': 'FAM_CD_pim'
}

pimfinal_new_df = pimfinal_new_df.rename(columns=new_column_names_pim)

In [54]:
pidfinal_new_df.head()

,CON_UPC_NO,SV_SV_FL,PV_FL,SLS_CHA_CD,PMY_DPT_CD,REC_DPT_CD,DPT_CD,COM_CD,SUBCOM_CD,SNAP_ELG_FL,RBP_GRP_ID,RBP_GRP_NAM,KMP_KLP_CD,CPT_BRN_PDT_CLS_CD,FAM_CD
0,0000000000001,,,NONE,03,03,03,186,69900,,,,,,
2,0000000000003,,,NONE,01,01,45,045,10206,F,,,,,
9,0000000000014,Yes,Yes,NONE,03,03,03,186,69900,,,,,,
10,0000000000015,Yes,Yes,STORE,01,43,14,047,04702,,,,,,
12,0000000000019,Yes,No,STORE,01,01,12,029,18217,F,,,,,


In [55]:
pimfinal_new_df.head()

,CON_UPC_NO,SV_SV_FL_pim,PV_FL_pim,SLS_CHA_CD_pim,PMY_DPT_CD_pim,REC_DPT_CD_pim,DPT_CD_pim,COM_CD_pim,SUBCOM_CD_pim,SNAP_ELG_FL_pim,RBP_GRP_ID_pim,RBP_GRP_NAM_pim,KMP_KLP_CD_pim,CPT_BRN_PDT_CLS_CD_pim,FAM_CD_pim
1,0000000000001,Yes,,,03,03,03,186,69900,,,,,,
3,0000000000003,Yes,,,01,01,45,045,10206,F,,,,,
4,0000000000014,Yes,Yes,,03,03,03,186,69900,,,,,,
5,0000000000015,No,No,,01,43,14,047,04702,,,,,,
8,0000000000019,Yes,Yes,STORE,01,01,12,029,18217,F,,,,,


In [56]:
# Print lengths for verification
print("Length of pidfinal_new_df:", len(pidfinal_new_df))
print("Length of pimfinal_new_df:", len(pimfinal_new_df))

Length of pidfinal_new_df: 1101248
Length of pimfinal_new_df: 1098190


In [57]:
pidfinal_new_df.to_csv("C:\\Users\\AS81970\\Desktop\\PID_Final_mismatch.csv", index=False)

In [58]:
pimfinal_new_df.to_csv("C:\\Users\\AS81970\\Desktop\\PIM_Final_mismatch.csv", index=False)

## Apply Compare Scenario Logic to build output File ##

In [59]:
%%time
# Merge the two dataframes on common columns
merged_df1 = pd.merge(pimfinal_new_df, pidfinal_new_df, on='CON_UPC_NO', how='inner')

# Define a function to apply the logic to create the columns
def apply_logic(row):
    sv_fl = ''
    if row['SV_SV_FL_pim'] == 'Yes' and row['SV_SV_FL'] == 'No':
        sv_fl = ''
    elif row['SV_SV_FL_pim'] == 'Yes' and row['SV_SV_FL'] == 'Yes':
        sv_fl = ''
    elif row['SV_SV_FL_pim'] == 'No' and row['SV_SV_FL'] == 'Yes':
        sv_fl = 'Yes'
    elif row['SV_SV_FL_pim'] == 'No' and row['SV_SV_FL'] == 'No':
        sv_fl = ''
    elif row['SV_SV_FL_pim'] == '' and row['SV_SV_FL'] == 'Yes':
        sv_fl = 'Yes'
    elif row['SV_SV_FL_pim'] == '' and row['SV_SV_FL'] == 'No':
        sv_fl = ''
    elif row['SV_SV_FL_pim'] == '' and row['SV_SV_FL'] == '':
        sv_fl = ''
    elif row['SV_SV_FL_pim'] == 'Yes' and row['SV_SV_FL'] == '':
        sv_fl = ''
    elif row['SV_SV_FL_pim'] == 'No' and row['SV_SV_FL'] == '':
        sv_fl = ''

    # Logic for PV_FL
    pv_fl = ''
    if row['PV_FL_pim'] == 'Yes' and row['PV_FL'] == 'No':
        pv_fl = ''
    elif row['PV_FL_pim'] == 'Yes' and row['PV_FL'] == 'Yes':
        pv_fl = ''
    elif row['PV_FL_pim'] == 'No' and row['PV_FL'] == 'Yes':
        pv_fl = 'Yes'
    elif row['PV_FL_pim'] == 'No' and row['PV_FL'] == 'No':
        pv_fl = ''
    elif row['PV_FL_pim'] == '' and row['PV_FL'] == 'Yes':
        pv_fl = 'Yes'
    elif row['PV_FL_pim'] == '' and row['PV_FL'] == 'No':
        pv_fl = 'No'
    elif row['PV_FL_pim'] == '' and row['PV_FL'] == '':
        pv_fl = ''
    elif row['PV_FL_pim'] == 'Yes' and row['PV_FL'] == '':
        pv_fl = ''
    elif row['PV_FL_pim'] == 'No' and row['PV_FL'] == '':
        pv_fl = ''       
        
    
    rbp_id = '' if row['RBP_GRP_ID_pim'] == row['RBP_GRP_ID'] else row['RBP_GRP_ID']
    rbp_desc = ''
    if row['RBP_GRP_ID_pim'] == row['RBP_GRP_ID'] and row['RBP_GRP_NAM_pim'] == row['RBP_GRP_NAM']:
        rbp_desc = ''
    elif row['RBP_GRP_ID_pim'] == row['RBP_GRP_ID'] and row['RBP_GRP_NAM_pim'] != row['RBP_GRP_NAM']:
        rbp_desc = row['RBP_GRP_NAM']
    elif row['RBP_GRP_ID_pim'] != row['RBP_GRP_ID']:
        rbp_desc = row['RBP_GRP_NAM']

    previous_rbd_id = '' if row['RBP_GRP_ID_pim'] == row['RBP_GRP_ID'] else row['RBP_GRP_ID_pim']
    previous_fam_cd = '' if row['FAM_CD_pim'] == row['FAM_CD'] else row['FAM_CD_pim']

    sales_cha = ''
    if row['SLS_CHA_CD_pim'] == 'MP; STORE' and row['SLS_CHA_CD'] == 'STORE':
        sales_cha = ''
    elif row['SLS_CHA_CD_pim'] == 'MP; STORE' and row['SLS_CHA_CD'] == 'STORE; DIRECT_SHIP':
        sales_cha = 'STORE;DIRECT_SHIP;MP'
    elif row['SLS_CHA_CD_pim'] == 'MP' and row['SLS_CHA_CD'] == 'STORE':
        sales_cha = 'STORE;MP'
    elif row['SLS_CHA_CD_pim'] == 'MP' and row['SLS_CHA_CD'] == 'NONE':
        sales_cha = 'MP'
    elif row['SLS_CHA_CD_pim'] == 'SHIP;STORE' and row['SLS_CHA_CD'] == 'NONE':
        sales_cha = 'NONE'
    elif row['SLS_CHA_CD_pim'] == 'SHIP;MP' and row['SLS_CHA_CD'] == 'NONE':
        sales_cha = 'MP'
    elif row['SLS_CHA_CD_pim'] == '' and row['SLS_CHA_CD'] == 'NONE':
        sales_cha = ''    
    elif row['SLS_CHA_CD_pim'] != row['SLS_CHA_CD']:
        sales_cha = row['SLS_CHA_CD']
        
    #ft_primary_dpt = '' if row['PMY_DPT_CD_pim'] == row['PMY_DPT_CD'] else row['PMY_DPT_CD']
    #ft_recap_dpt = '' if row['REC_DPT_CD_pim'] == row['REC_DPT_CD'] else row['REC_DPT_CD']
    #ft_dpt = '' if row['DPT_CD_pim'] == row['DPT_CD'] else row['DPT_CD']
    #ft_comm_cd = '' if row['COM_CD_pim'] == row['COM_CD'] else row['COM_CD']
    #ft_subcom = '' if row['SUBCOM_CD_pim'] == row['SUBCOM_CD'] else row['SUBCOM_CD']
    
    snap_fl = '' if row['SNAP_ELG_FL_pim'] == row['SNAP_ELG_FL'] else row['SNAP_ELG_FL']
    klp_cd = '' if row['KMP_KLP_CD_pim'] == row['KMP_KLP_CD'] else row['KMP_KLP_CD']
    pdt_cd = '' if row['CPT_BRN_PDT_CLS_CD_pim'] == row['CPT_BRN_PDT_CLS_CD'] else row['CPT_BRN_PDT_CLS_CD']
    fam_cd = '' if row['FAM_CD_pim'] == row['FAM_CD'] else row['FAM_CD']
    
    ft_primary_dpt = '' if (
    row['PMY_DPT_CD_pim'] == row['PMY_DPT_CD'] and
    row['REC_DPT_CD_pim'] == row['REC_DPT_CD'] and
    row['DPT_CD_pim'] == row['DPT_CD'] and
    row['COM_CD_pim'] == row['COM_CD'] and
    row['SUBCOM_CD_pim'] == row['SUBCOM_CD']
    ) else row['PMY_DPT_CD']
    
    ft_recap_dpt = '' if (
    row['PMY_DPT_CD_pim'] == row['PMY_DPT_CD'] and
    row['REC_DPT_CD_pim'] == row['REC_DPT_CD'] and
    row['DPT_CD_pim'] == row['DPT_CD'] and
    row['COM_CD_pim'] == row['COM_CD'] and
    row['SUBCOM_CD_pim'] == row['SUBCOM_CD']
    ) else row['REC_DPT_CD']
        
    ft_dpt = '' if (
    row['PMY_DPT_CD_pim'] == row['PMY_DPT_CD'] and
    row['REC_DPT_CD_pim'] == row['REC_DPT_CD'] and
    row['DPT_CD_pim'] == row['DPT_CD'] and
    row['COM_CD_pim'] == row['COM_CD'] and
    row['SUBCOM_CD_pim'] == row['SUBCOM_CD']
    ) else row['DPT_CD']  

    ft_comm_cd = '' if (
    row['PMY_DPT_CD_pim'] == row['PMY_DPT_CD'] and
    row['REC_DPT_CD_pim'] == row['REC_DPT_CD'] and
    row['DPT_CD_pim'] == row['DPT_CD'] and
    row['COM_CD_pim'] == row['COM_CD'] and
    row['SUBCOM_CD_pim'] == row['SUBCOM_CD']
    ) else row['COM_CD']  

    ft_subcom = '' if (
    row['PMY_DPT_CD_pim'] == row['PMY_DPT_CD'] and
    row['REC_DPT_CD_pim'] == row['REC_DPT_CD'] and
    row['DPT_CD_pim'] == row['DPT_CD'] and
    row['COM_CD_pim'] == row['COM_CD'] and
    row['SUBCOM_CD_pim'] == row['SUBCOM_CD']
    ) else row['SUBCOM_CD']   

    return pd.Series({
        'CON_UPC_NO': row['CON_UPC_NO'],
        'SV_FL': sv_fl, 
        'PV_FL': pv_fl,
        'RBP_ID': rbp_id,
        'RBP_DESC': rbp_desc,
        'PREVIOUS_RBD_ID': previous_rbd_id,
        'PREVIOUS_FAM_CD': previous_fam_cd,
        'SALES_CHA': sales_cha,
        'FT_PRIMARY_DPT': ft_primary_dpt,
        'FT_RECAP_DPT': ft_recap_dpt,
        'FT_DPT': ft_dpt,
        'FT_COMM_CD': ft_comm_cd,
        'FT_SUBCOM': ft_subcom,
        'SNAP_FL': snap_fl, 
        'KLP_CD': klp_cd,
        'PDT_CD': pdt_cd,
        'FAM_CD': fam_cd
    })

# Apply the function to each row of the merged dataframe
new_df = merged_df1.apply(apply_logic, axis=1)

# Display the new dataframe
new_df.head()

CPU times: total: 9min 18s
Wall time: 9min 21s


,CON_UPC_NO,SV_FL,PV_FL,RBP_ID,RBP_DESC,PREVIOUS_RBD_ID,PREVIOUS_FAM_CD,SALES_CHA,FT_PRIMARY_DPT,FT_RECAP_DPT,FT_DPT,FT_COMM_CD,FT_SUBCOM,SNAP_FL,KLP_CD,PDT_CD,FAM_CD
0,0000000000001,,,,,,,,,,,,,,,,
1,0000000000003,,,,,,,,,,,,,,,,
2,0000000000014,,,,,,,,,,,,,,,,
3,0000000000015,Yes,Yes,,,,,STORE,,,,,,,,,
4,0000000000019,,,,,,,,,,,,,,,,


In [60]:
#if all cloumns blank, drop that row.
# Convert empty strings to NaN
new_df.replace('', pd.NA, inplace=True)

# Drop rows where all values for that UPC (CON_UPC_NO) are empty
new_df.dropna(subset=new_df.columns[1:], how='all', inplace=True)

new_df.head()

,CON_UPC_NO,SV_FL,PV_FL,RBP_ID,RBP_DESC,PREVIOUS_RBD_ID,PREVIOUS_FAM_CD,SALES_CHA,FT_PRIMARY_DPT,FT_RECAP_DPT,FT_DPT,FT_COMM_CD,FT_SUBCOM,SNAP_FL,KLP_CD,PDT_CD,FAM_CD
3,0000000000015,Yes,Yes,<NA>,<NA>,<NA>,<NA>,STORE,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
5,0000000000023,<NA>,No,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
6,0000000000024,<NA>,No,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
7,0000000000025,<NA>,No,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
8,0000000000029,<NA>,No,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [61]:
num_rows = len(new_df)
print("Number of rows:", num_rows)

Number of rows: 354600


In [62]:
#Write the PID new_df DataFrame to CSV #300k important!!
new_df.to_csv("C:\\Users\\AS81970\\Desktop\\new_df.csv", index=False)

## compare Tester ouput vs Developer output and show differences! ##

In [63]:
new_df_File = "C:\\Users\\AS81970\\Desktop\\new_df.csv"

new_df_File = pd.read_csv(new_df_File, sep=',', dtype=str,header=0, encoding='latin-1')

desired_order = ["CON_UPC_NO", "SALES_CHA", "SV_FL", "FT_PRIMARY_DPT", "FT_RECAP_DPT", "FT_DPT", "FT_COMM_CD", "FT_SUBCOM", "SNAP_FL", "RBP_ID", "RBP_DESC", "PREVIOUS_RBD_ID", "PV_FL", "KLP_CD", "PDT_CD", "FAM_CD", "PREVIOUS_FAM_CD"]

new_df_File = new_df_File[desired_order]

original_new_df_File = new_df_File.copy()

with open("C:\\Users\\AS81970\\Desktop\\new_df.csv") as file:
    num_lines_newdf = original_new_df_File.shape[0]
    print(f"Number of lines: {num_lines_newdf}")

new_df_File.head()

Number of lines: 354600


,CON_UPC_NO,SALES_CHA,SV_FL,FT_PRIMARY_DPT,FT_RECAP_DPT,FT_DPT,FT_COMM_CD,FT_SUBCOM,SNAP_FL,RBP_ID,RBP_DESC,PREVIOUS_RBD_ID,PV_FL,KLP_CD,PDT_CD,FAM_CD,PREVIOUS_FAM_CD
0,0000000000015,STORE,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN
1,0000000000023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
2,0000000000024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
3,0000000000025,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN
4,0000000000029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN


In [64]:
Sync_Dev_File = "C:\\Users\\AS81970\\Desktop\\2023 Q2\\Itemchanges_SalesChannel_SV_FT.csv"

Sync_dev_df = pd.read_csv(Sync_Dev_File, sep='|', dtype=str,header=0, encoding='latin-1')

# Convert 14-digit GTIN to 13-digit UPC
Sync_dev_df['GTIN_NO'] = Sync_dev_df['GTIN_NO'].str.slice(stop=-1)

# Add a suffix to all columns except 'GTIN_NO'
suffix = '_dev'  # Replace 'suffix' with your desired suffix
Sync_dev_df.rename(columns=lambda x: x + suffix if x != 'GTIN_NO' else x, inplace=True)

# Change values in 'PV_FLAG_dev' column from 'Yes' to 'Y' and 'No' to 'N'
Sync_dev_df['PV_FLAG_dev'] = Sync_dev_df['PV_FLAG_dev'].replace({'Y': 'Yes', 'N': 'No'})

# Change values in 'SALES_VARIANT_dev' column from 'YES' to 'Yes' and 'NO' to 'No'
Sync_dev_df['SALES_VARIANT_dev'] = Sync_dev_df['SALES_VARIANT_dev'].replace({'YES': 'Yes', 'NO': 'No'})

original_sync_df_File = Sync_dev_df.copy()

with open("C:\\Users\\AS81970\\Desktop\\2023 Q2\\Itemchanges_SalesChannel_SV_FT.csv") as file:
    num_lines = original_sync_df_File.shape[0]
    print(f"Number of lines: {num_lines}")

Sync_dev_df.head()

Number of lines: 15772


,GTIN_NO,SALES_CHANNEL_dev,SALES_VARIANT_dev,PRIMARY_dev,RECAP_dev,DEPT_dev,COMMODITY_dev,SUBCOMMODITY_dev,SNAP_dev,RBP_GRP_ID_dev,...,PCS_MDE_TYP_CD_dev,CON_PRO_DSC_TX_dev,EAS_CD_dev,EAS_TAG_PDT_PN1_dev,PRO_PAK_FL1_dev,CPT_INF_CD_dev,EAS_EFF_DT_dev,UNT_PRC_SIZ_CD_dev,UNT_FTR_AM_dev,UNT_OVD_CD_dev
0,0000000000085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.00,NaN
1,0000000000690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0000000000693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0000000000695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0000000000921,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.00,NaN


In [65]:
# REPORT#

# Find the UPCs missing in new_df_File but present in Sync_dev_df
missing_in_new_df = set(Sync_dev_df['GTIN_NO']) - set(new_df_File['CON_UPC_NO'])

# Create a DataFrame for missing UPCs from new_df_File
missing_new_df = pd.DataFrame({'Missing_UPC': list(missing_in_new_df)})

# Find the UPCs missing in Sync_dev_df but present in new_df_File
missing_in_sync_dev = set(new_df_File['CON_UPC_NO']) - set(Sync_dev_df['GTIN_NO'])

# Create a DataFrame for missing UPCs from new_df_File
missing_sync_dev = pd.DataFrame({'Missing_UPC': list(missing_in_sync_dev)})

count_missing_new_df = len(missing_in_new_df)
count_missing_sync_dev = len(missing_in_sync_dev)

In [66]:
# Find the intersection of UPCs between new_df_File and Sync_dev_df
Intersection_upcs = set(new_df_File['CON_UPC_NO']).intersection(set(Sync_dev_df['GTIN_NO']))

# Create a DataFrame for intersection UPCs from both files
Intersection_upcs_df = pd.DataFrame({'Intersection upcs': list(Intersection_upcs)})
count_Intersection_upcs_df = len(Intersection_upcs_df)

# Keep only the matching rows in both dataframes
new_df_File = new_df_File[new_df_File['CON_UPC_NO'].isin(Intersection_upcs)].copy()
Sync_dev_df = Sync_dev_df[Sync_dev_df['GTIN_NO'].isin(Intersection_upcs)].copy()

# Sort both dataframes by their UPC columns
new_df_File.sort_values(by='CON_UPC_NO', inplace=True)
Sync_dev_df.sort_values(by='GTIN_NO', inplace=True)

In [67]:
# Your column mapping
column_mapping = {
    #"CON_UPC_NO": "GTIN_NO",  # Mapping based on the matching columns
    "SV_FL": "SALES_VARIANT_dev",
    "PV_FL": "PV_FLAG_dev",
    "RBP_ID": "RBP_GRP_ID_dev",
    "RBP_DESC": "RBP_GRP_DESC_dev",
    "PREVIOUS_RBD_ID": "OLD_RBP_GRP_ID_dev",
    "PREVIOUS_FAM_CD": "PREV_FAM_CD_dev",
    "SALES_CHA": "SALES_CHANNEL_dev",
    "FT_PRIMARY_DPT": "PRIMARY_dev",
    "FT_RECAP_DPT": "RECAP_dev",
    "FT_DPT": "DEPT_dev",
    "FT_COMM_CD": "COMMODITY_dev",
    "FT_SUBCOM": "SUBCOMMODITY_dev",
    "SNAP_FL": "SNAP_dev",
    "KLP_CD": "KLP_KMP_CD_dev",
    "PDT_CD": "PDT_CLS_CD_dev",
    "FAM_CD": "FAM_CD_dev"
}

# Merge the two DataFrames on 'GTIN_NO' and 'CON_UPC_NO'
merged_out2_df = new_df_File.merge(Sync_dev_df, left_on='CON_UPC_NO', right_on='GTIN_NO', how='inner')

def compare_columns(row):
    for new_col, sync_col in column_mapping.items():
        val_new = row[new_col]       # Value in the corresponding column of new_df_File
        val_sync = row[sync_col]     # Value in the corresponding column of Sync_dev_df

        if pd.isna(val_new) and pd.isna(val_sync):
            continue
        elif val_new == val_sync:
            continue
        else:
            return 'Mismatch'
    return 'Matched'

# Create the 'Status' column based on the comparison
merged_out2_df['Status'] = merged_out2_df.apply(compare_columns, axis=1)

# Specify the desired column order
desired_column_order = [
    'CON_UPC_NO', 'SALES_CHA', 'SV_FL', 'FT_PRIMARY_DPT', 'FT_RECAP_DPT', 'FT_DPT', 'FT_COMM_CD', 'FT_SUBCOM',
    'SNAP_FL', 'RBP_ID', 'RBP_DESC', 'PREVIOUS_RBD_ID', 'PV_FL', 'KLP_CD', 'PDT_CD', 'FAM_CD', 'PREVIOUS_FAM_CD',
    'Status', 'GTIN_NO', 'SALES_CHANNEL_dev', 'SALES_VARIANT_dev', 'PRIMARY_dev', 'RECAP_dev', 'DEPT_dev',
    'COMMODITY_dev', 'SUBCOMMODITY_dev', 'SNAP_dev', 'RBP_GRP_ID_dev', 'RBP_GRP_DESC_dev', 'OLD_RBP_GRP_ID_dev',
    'RBP_CHANGE_DATE_dev', 'PV_FLAG_dev', 'KLP_KMP_CD_dev', 'PDT_CLS_CD_dev', 'FAM_CD_dev', 'PREV_FAM_CD_dev',
]

# Reorder columns in merged_df2
merged_out2_df = merged_out2_df[desired_column_order]

merged_out2_df.head()

# Calculate the count of 'Mismatch' and 'Matched' statuses
status_counts = merged_out2_df['Status'].value_counts()

In [68]:
merged_out2_df.to_csv("C:\\Users\\AS81970\\Desktop\\Comparisson.csv", index=False)

# Summary Report

In [69]:
with open("C:\\Users\\AS81970\\Desktop\\new_df.csv") as file:
    num_lines_newdf = original_new_df_File.shape[0]
    print(f"Number of Records in Tester File: {num_lines_newdf}")

with open("C:\\Users\\AS81970\\Desktop\\2023 Q2\\Itemchanges_SalesChannel_SV_FT.csv") as file:
    num_lines = original_sync_df_File.shape[0]
    print(f"Number of Records in Developer File: {num_lines}")

print("Count of UPCs in Intersection:", count_Intersection_upcs_df)
    
print("UPCs NOT in Tester File:", count_missing_new_df)
print("UPCs NOT in Developer File:", count_missing_sync_dev)

#Print the status counts
print("Mismatch Count:", status_counts['Mismatch'])
print("Matched Count:", status_counts['Matched'])

# Write the missing UPCs to separate CSV files
missing_new_df.to_csv("C:\\Users\\AS81970\\Desktop\\missing_in_Tester_File.csv", index=False)
missing_sync_dev.to_csv("C:\\Users\\AS81970\\Desktop\\missing_in_Dev_File.csv", index=False)

Number of Records in Tester File: 354600
Number of Records in Developer File: 15772
Count of UPCs in Intersection: 4772
UPCs NOT in Tester File: 11000
UPCs NOT in Developer File: 349097
Mismatch Count: 1776
Matched Count: 3001
